# 基于驱动级别模块pywinio的键盘自动化
---

## 一、使用前注意事项
**运行以下模块的程序必须以管理员身份运行！不然会报错拒绝访问！**

## 二、具体代码

In [2]:
#!/usr/bin/env python
# -*- coding:utf-8 -*-
#author:June_xu

#模块介绍：
# pywinio win10或以上系统需要接上ps / 2键盘，也就是圆头接口的键盘，实际测试并不需要。。
#本代码采用驱动级别模块pywinio 此模块前身名叫rabird.winio，如果需要在网上搜索资料请是使用这个名称。
#按键扫描码存储在三个字典中。但是内容不全。
#关于字母大小写：inputWords中能够自动识别字符串和大小写，但是press_down和press_up是输入键盘扫描码的。

import pywinio
import time,random
import atexit

#-----------------------------------按键扫描码
#参考链接https://blog.csdn.net/qq_37232329/article/details/79926440。
#文章二：https://www.cnblogs.com/love-zf/p/5093881.html
# 文章中有些特殊按键带有多个扫描码，也就是使用ke_down先生成一个扫描码，再key_down生成另一个，最后key_up
#蛛键盘按键
keyboard_scanCode={
    '0':0x0b,'1':0x02,'2':0x03,'3':0x04,'4':0x05,'5':0x06,'6':0x07,'7':0x08,'8':0x09,'9':0x0a,
    ',':0x0333,'.':0x0534,'/':0x0635,'*':0x0837,' ':0x0a39,'shift':0x2a,'ctrl':0x1d,'alt':0x38,'tab':0x0f,
    'a':0x1e,'b':0x30,'c':0x2e,'d':0x20,'e':0x12,'f':0x21,'g':0x22,'h':0x23,'i':0x17,'j':0x24,'k':0x25,'l':0x26,
    'm':0x32,'n':0x31,'o':0x18,'p':0x19,'q':0x10,'r':0x13,'s':0x1f,'t':0x14,'u':0x16,'v':0x2f,
    'w':0x11,'x':0x2d,'y':0x15,'z':0x2c,
}
#按住shift才会出现的按键，
keyboard_shift_scanCode={
    "!":0x02,"@":0x03,'#':0x04,'$':0x05,'%':0x06,'^':0x07,'&':0x08,'*':0x09,'(':0x0a,')':0x0b
}
#小键盘按键
smallKeyboard_scanCode={
'0':0x0b30,'1':0x4f,'2':0x50,'3':0x51,'4':0x4b,'5':0x4c,'6':0x4d,'7':0x47,'8':0x48,'9':0x49,'num':0x45,'.':0xc5
}#如果关闭小键盘，0xc5则是删除。和键盘上的标记一样。

keyboard_special_scanCode={
    'del':[0xe0,0x53],'PrintScreen':[0xe0,0x2a,0xe0,0x37],
    'left':[0xe0,0x46],'right':[0xe0,0x4d],'up':[0xe0,0x48],'down':[0xe0,0x50],
    'pageDown':[0xe0,0x51],'pageUp':[0xe0,0x49]
}#此处部分功能未做测试


#------------------------------------主代码
KBC_KEY_CMD = 0x64
# Data port
KBC_KEY_DATA = 0x60

__winio = None
def __get_winio():
    global __winio

    if __winio is None:
            __winio = pywinio.WinIO()
            def __clear_winio():
                    global __winio
                    __winio = None
            atexit.register(__clear_winio)

    return __winio

def wait_for_buffer_empty():
    '''
    Wait keyboard buffer empty
    '''

    winio = __get_winio()

    dwRegVal = 0x02
    while (dwRegVal & 0x02):
            dwRegVal = winio.get_port_byte(KBC_KEY_CMD)

#按下按键
def key_down(scancode):
    winio = __get_winio()

    wait_for_buffer_empty()
    winio.set_port_byte(KBC_KEY_CMD, 0xd2)
    wait_for_buffer_empty()
    winio.set_port_byte(KBC_KEY_DATA, scancode)

def key_up(scancode):
    winio = __get_winio()

    wait_for_buffer_empty()
    winio.set_port_byte( KBC_KEY_CMD, 0xd2)
    wait_for_buffer_empty()
    winio.set_port_byte( KBC_KEY_DATA, scancode | 0x80)

def key_press(scancode, press_time = 0.2):
    key_down( scancode )
    time.sleep( press_time )
    key_up( scancode )


def inputWords(text,isDelay=True):
    '''

    :param text: 文字内容
    :param isDelay: 按键随机延时，主要用于防按键检测，一般情况下用不到
    :return:
    '''
    global keyboard_scanCode, keyboard_shift_scanCode
    def shiftInput(scancode):
        try:
            key_down(keyboard_scanCode["shift"])
            time.sleep(0.2)
            key_press(scancode)
        finally:
            key_up(keyboard_scanCode["shift"])
            time.sleep(0.2)

    def specialInput(scanCodeList):
        try:
            for scanCode in scanCodeList:
                key_down(scanCode)
        finally:
            for scanCode in scanCodeList:
                key_up(scanCode)

    for i in text:
        print(i)
        upper=False
        if i.isupper() == True:
            i = i.lower()
            print("大写字母修正,shift+%s" % i)
            upper=True

        if i in keyboard_scanCode:
            if upper==True:
                shiftInput(keyboard_scanCode[i])
            else:
                key_press(keyboard_scanCode[i])
        elif i in keyboard_shift_scanCode:
            shiftInput(keyboard_shift_scanCode[i])
            time.sleep(0.5)
        elif i in smallKeyboard_scanCode:
            key_press(smallKeyboard_scanCode[i])
        elif i in keyboard_special_scanCode:
            specialInput(keyboard_special_scanCode[i])
        else:
            raise ValueError("%s的值不在keybaord扫描码中，需要手动添加"%i)

        if isDelay==True:
            time.sleep(round(random.randint(200,500)/1000,1))

## 三、使用方法
**单个按键输入**
<code>
    key_down(keyboard_scanCode['h'])#按下
    key_up(keyboard_scanCode['h'])#弹起
</code>
**或者**
<code>
    key_press(keyboard_scanCode['h'])
</code>

**输入连续的字符串**
<code>
    inputWords('hello',False)
</code>